In [1]:
import numpy as np
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import keras
import pandas as pd
import queue
import math
import cv2
from collections import deque

2023-04-25 10:17:29.478517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 10:17:30.266512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# model load
model = keras.models.load_model('/home/siwon/dev/Deeplearning-6/model/first_model.h5')

# test video load
test_video_path = '/home/siwon/dev/Deeplearning-6/data_video/test_video1.mp4'
test_video = cv2.VideoCapture(test_video_path)

2023-04-25 10:17:31.313313: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 10:17:31.335114: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 10:17:31.335507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
cap = cv2.VideoCapture(-1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
# 궤적 리스트
position_list = deque(maxlen=5)
trajectory_list = []
append_on = False
t=0

# 주어진 입력 데이터에서 연속적으로 다음 위치 예측 함수
def predict_next_positions(model, input_data, num_steps):
    current_input = input_data.copy()
    predicted_positions = []

    for _ in range(num_steps):
        predicted = model.predict(current_input)
        predicted_positions.append(predicted[0])

        # 처음 4개 좌표와 예측한 좌표로 새로운 입력 데이터 생성
        new_input = np.concatenate((current_input[:, 1:, :], predicted.reshape(1, 1, -1)), axis=1)
        current_input = new_input

    return np.array(predicted_positions)


while True:
    # 프레임을 읽어옵니다.
    ret, frame = cap.read()
    # HSV 색공간으로 변환합니다.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # 주황색 범위를 정의합니다.
    lower_orange = np.array([5, 70, 200])
    upper_orange = np.array([100, 255, 255])
    # 주황색 영역을 마스크로 만듭니다.
    mask = cv2.inRange(hsv, lower_orange, upper_orange)

    # 모폴로지 연산을 사용하여 마스크를 개선합니다.
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.erode(mask, kernel)
    mask = cv2.dilate(mask, kernel)
    
    # 주황색 영역을 바탕으로 객체의 윤곽선을 추출합니다.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # 가장 큰 윤곽선을 추출합니다.
    if len(contours) > 0:
        c = max(contours, key=cv2.contourArea)
        
        # 윤곽선을 둘러싸는 사각형을 그립니다.
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        
        # 중심 좌표를 계산합니다.
        center_x = x + w//2
        center_y = y + h//2

        
        # 72X72 격자 모델
        # 중심 좌표 x,y를 프레임에 출력합니다.
        if center_x > 420 and center_x < 1500:
            append_on = True
            cv2.putText(frame, "x: " + str(center_x), (center_x + 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "y: " + str(center_y), (center_x + 10, center_y + 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            new_position = [math.ceil((center_x - 420) / 15), math.ceil(center_y / 15)]
            if new_position not in position_list:
                new_position.append(t)
                position_list.append(new_position)
                t= t+1
                #print(position_list)
            # model predict
            if len(position_list) == 5:
                predict_data = np.array(position_list)
                predict_data = predict_data.reshape(1, 5, 3)
                predict_result = predict_next_positions(model, predict_data, 5)
                ## marker predict
                for i in range(len(predict_result)):
                    cv2.circle(frame,((15*(round(predict_result[i][0])))+420, 15*(round(predict_result[i][1]))), 10, (0, 0, 255), -1)
                
        else:
            append_on = False
            if len(position_list) > 0:
                trajectory_list.append(position_list)
                position_list = deque(maxlen=5)
        if t >= 50:
            position_list.clear()
            t = 0

    # 72X72 격자 모델
    # 격자 무늬를 출력합니다.
    for i in range(420, 1515, 15):
        cv2.line(frame, (i, 0), (i, 1080), (255, 0, 0), 1)
    for i in range(0, 1095, 15):
        cv2.line(frame, (420, i), (1500, i), (255, 0, 0), 1)

    # 사각형 그리기
    cv2.rectangle(frame, (420, 0), (1500, 1080), (0, 255, 0), 2)
    # 결과를 출력합니다.
    cv2.imshow('Orange Ball Tracker', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 22ms/step
